In [1]:
from sklearn.feature_extraction.text import CountVectorizer

In [55]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PowerTransformer 
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.metrics import mean_squared_error
from sklearn import set_config
from sklearn.linear_model import LinearRegression, Ridge, Lasso


In [56]:
#pd.set_option('display.max_rows', None)

In [57]:
X_train = pd.read_csv('X_train.csv', encoding='cp949').drop(columns='ID')
y_train = pd.read_csv('y_train.csv', encoding='cp949').Salary

X_test = pd.read_csv('X_test.csv', encoding='cp949')
test_id = X_test.ID
X_test = X_test.drop(columns='ID')

In [58]:
def resumetable(df):
    print(f'데이터셋 형상: {df.shape}')
    summary = pd.DataFrame(df.dtypes, columns=['데이터 타입'])
    summary = summary.reset_index()
    summary = summary.rename(columns={'index': '피처'})
    summary['결측값 개수'] = df.isnull().sum().values
    summary['고윳값 개수'] = df.nunique().values
    summary['첫 번째 값'] = df.loc[0].values
    summary['두 번째 값'] = df.loc[1].values
    summary['세 번째 값'] = df.loc[2].values
    
    return summary


In [59]:
feature_resume_table = resumetable(X_train)
feature_resume_table

데이터셋 형상: (16570, 11)


,피처,데이터 타입,결측값 개수,고윳값 개수,첫 번째 값,두 번째 값,세 번째 값
0,직종,object,0,11,문화·예술·신문·방송,경영·기획·회계·사무,IT·게임
1,세부직종,object,0,109,영상·음향·사진·카메라,사무·총무·법무,하드웨어설계·개발·관리
2,직무태그,object,2515,11815,"취재기자, 편집 기사, 유아 사이트 기획, 시나리오 작성",NaN,"하드웨어 , 무선통신 , MICOM , ASM , RF , CDMA , Firm W..."
3,근무경력,object,0,311,0개월,2년 11개월,0개월
4,근무형태,object,9909,46,NaN,정규직,NaN
5,근무지역,object,0,1313,"서울,경기,서울","부산,서울,일본","서울,경기,"
6,출신대학,object,0,41,성균관대학교,신라대학교,수원대학교
7,대학전공,object,0,2030,국문,관광경영,정보통신공
8,어학시험,object,11582,26,NaN,JLPT,NaN
9,자격증,object,0,2,無,無,有


In [60]:
feature_resume_table = resumetable(X_train)
feature_resume_table

데이터셋 형상: (16570, 11)


,피처,데이터 타입,결측값 개수,고윳값 개수,첫 번째 값,두 번째 값,세 번째 값
0,직종,object,0,11,문화·예술·신문·방송,경영·기획·회계·사무,IT·게임
1,세부직종,object,0,109,영상·음향·사진·카메라,사무·총무·법무,하드웨어설계·개발·관리
2,직무태그,object,2515,11815,"취재기자, 편집 기사, 유아 사이트 기획, 시나리오 작성",NaN,"하드웨어 , 무선통신 , MICOM , ASM , RF , CDMA , Firm W..."
3,근무경력,object,0,311,0개월,2년 11개월,0개월
4,근무형태,object,9909,46,NaN,정규직,NaN
5,근무지역,object,0,1313,"서울,경기,서울","부산,서울,일본","서울,경기,"
6,출신대학,object,0,41,성균관대학교,신라대학교,수원대학교
7,대학전공,object,0,2030,국문,관광경영,정보통신공
8,어학시험,object,11582,26,NaN,JLPT,NaN
9,자격증,object,0,2,無,無,有


In [61]:
feature_resume_table = resumetable(X_test)
feature_resume_table

데이터셋 형상: (11048, 11)


,피처,데이터 타입,결측값 개수,고윳값 개수,첫 번째 값,두 번째 값,세 번째 값
0,직종,object,0,12,전문·교육·자격,영업·판매·TM,건설·기계·전기·전자
1,세부직종,object,0,106,외국어·번역·통역,해외영업,기계·기계설계·기계설비
2,직무태그,object,1694,7978,"중국어,영어,강사,사무,micro office,호텔서비스",해외영업,"자동차,조선품질보증,품질관리,기계,산업기계,프레스,기계설비"
3,근무경력,object,0,301,0개월,0개월,15년 9개월
4,근무형태,object,6697,39,NaN,NaN,"정규직, 해외취업"
5,근무지역,object,0,1101,"서울,중국,캐나다","서울,,","부산,경남,경북"
6,출신대학,object,0,42,이화여자대학교,중앙대학교,동의대학교
7,대학전공,object,0,1622,증어중문,경영학과,산업공학과
8,어학시험,object,7706,25,TOEFL,,TOEIC
9,자격증,object,0,2,無,有,有


In [62]:
X_train.isnull().sum()

직종          0
세부직종        0
직무태그     2515
근무경력        0
근무형태     9909
근무지역        0
출신대학        0
대학전공        0
어학시험    11582
자격증         0
대학성적     1970
dtype: int64

In [63]:
print(X_train.isnull().sum() / len(X_train)*100)

직종       0.000000
세부직종     0.000000
직무태그    15.178033
근무경력     0.000000
근무형태    59.800845
근무지역     0.000000
출신대학     0.000000
대학전공     0.000000
어학시험    69.897405
자격증      0.000000
대학성적    11.888956
dtype: float64


In [64]:
print(X_test.isnull().sum() / len(X_test)*100)

직종       0.000000
세부직종     0.000000
직무태그    15.333092
근무경력     0.000000
근무형태    60.617306
근무지역     0.000000
출신대학     0.000000
대학전공     0.000000
어학시험    69.750181
자격증      0.000000
대학성적    11.929761
dtype: float64


### 범주형 변수

#### 직종, 세부직종, 직무태그, 근무형태

In [65]:
cat_features = X_train[['직종','세부직종','직무태그','근무형태']]

##### cardinality 개수

In [66]:
len(cat_features.직종.value_counts())

11

In [67]:
len(cat_features.세부직종.value_counts())

109

In [68]:
len(cat_features.직무태그.value_counts())

11815

## 근무형태 전처리

In [69]:
X_train['근무형태'].fillna('비정규직', inplace=True)

X_test['근무형태'].fillna('비정규직', inplace=True)

In [70]:
X_train['근무형태'] = X_train['근무형태'].str.replace(',', ' ')
X_train['근무형태'] = X_train['근무형태'].str.replace(' ', '')

X_test['근무형태'] = X_test['근무형태'].str.replace(',', ' ')
X_test['근무형태'] = X_test['근무형태'].str.replace(' ', '')

#### 근무지역, 출신대학, 대학전공, 어학시험, 자격증

## 어학시험 전처리

- 결측값은 자격증을 없는 것으로 간주하고 '없음'으로 채워넣음.

In [71]:
len(X_train.어학시험.value_counts())

26

In [72]:
X_train.어학시험 = X_train.어학시험.fillna('없음')

X_test.어학시험 = X_test.어학시험.fillna('없음')

In [73]:
X_train.어학시험.isnull().sum()

0

In [74]:
X_test.어학시험.isnull().sum()

0

## 자격증 전처리

In [75]:
X_train.자격증.value_counts()

無    8952
有    7618
Name: 자격증, dtype: int64

In [76]:
X_test.자격증.value_counts()

無    5948
有    5100
Name: 자격증, dtype: int64

In [77]:
X_train.자격증.fillna(X_train['자격증'].mode()[0], inplace=True)

X_test.자격증.fillna(X_test['자격증'].mode()[0], inplace=True)

In [78]:
X_train['자격증'] = X_train['자격증'].map({'有': 1, '無': 0})

X_test['자격증'] = X_test['자격증'].map({'有': 1, '無': 0})

In [79]:
X_train.isnull().sum()

직종         0
세부직종       0
직무태그    2515
근무경력       0
근무형태       0
근무지역       0
출신대학       0
대학전공       0
어학시험       0
자격증        0
대학성적    1970
dtype: int64

### 수치형 변수

In [80]:
X_train.대학성적.fillna(X_train['대학성적'].mode()[0], inplace = True)

X_test.대학성적.fillna(X_test['대학성적'].mode()[0], inplace = True)

In [81]:
for df in [X_train, X_test]:
    for i in range(len(df)):
        if len(df['근무경력'][i]) == 3:
            df['근무경력'][i] = int(df['근무경력'][i][0])
        elif len(df['근무경력'][i]) == 4:
            df['근무경력'][i] = int(df['근무경력'][i][:1])
        else:
            df['근무경력'][i] = int(df['근무경력'][i].split(' ')[0][:-1])*12 + int(df['근무경력'][i].split(' ')[1][:-2])

In [82]:
X_train['대학성적'].max()

110.0

In [83]:
X_train.isnull().sum()

직종         0
세부직종       0
직무태그    2515
근무경력       0
근무형태       0
근무지역       0
출신대학       0
대학전공       0
어학시험       0
자격증        0
대학성적       0
dtype: int64

In [84]:
X_test.isnull().sum()

직종         0
세부직종       0
직무태그    1694
근무경력       0
근무형태       0
근무지역       0
출신대학       0
대학전공       0
어학시험       0
자격증        0
대학성적       0
dtype: int64

----

# feature 생성

# 대학전공_대분류

In [85]:
X_train5 = X_train.copy()
X_test5 = X_test.copy()
y_train5 = y_train.copy()

In [86]:
X = pd.concat([X_train5,X_test5], ignore_index=True)

In [87]:
X['대학전공s5_대분류'] = X['대학전공'].copy()

In [88]:
# 문과
X['대학전공s5_대분류'][X['대학전공s5_대분류'].str.contains('경영|경제|경역')] = '경영학과'
X['대학전공s5_대분류'][X['대학전공s5_대분류'].str.contains('비즈니스|마케팅|Business|BUSINESS|business|e-busi|파이낸셜|비지니스|호텔|Management|HOTEL|유통|비서|MIS|상학과|관경|Trade|미국통상|관좡학과')] = '비즈니스'
X['대학전공s5_대분류'][X['대학전공s5_대분류'].str.contains('회계|전산|계산|상경|금융|재무관리')] = '재무회계'

X['대학전공s5_대분류'][X['대학전공s5_대분류'].str.contains('영미|문예|국어|독어|영어|일어|불어|중어|영문|일본|중문|노어|프랑스|국문|어문|일문|불문|러시아|스페인|한문|중국|language|독문|언어|노문|관공통역|독일')] = '어문대학'
X['대학전공s5_대분류'][X['대학전공s5_대분류'].str.contains('인류|철학|사학|사하|사료|윤리|기독교|인문|행정|신학|선교어문|민속|선교|북한|한국|목회|역사|국사|library & info. science')] = '인문대학'
X['대학전공s5_대분류'][X['대학전공s5_대분류'].str.contains('사회|통계|정치|무역|심리|아동|부동산|관광|보험|도서관|국제|지리|교정|Administration|정외|가정|노년|복지|지역|경찰|경호|청소년|소비자|보육|주거|주생활|산업재산권|핼정')] = '사회과학대학'

X['대학전공s5_대분류'][X['대학전공s5_대분류'].str.contains('광고|미디어|홍보|콘텐츠|방송|매체|문화|컨텐츠|Communication Consulting|경매컨설팅|신문|광보흥보')] = '미디어'
X['대학전공s5_대분류'][X['대학전공s5_대분류'].str.contains('교육|education|사범')] = '교육대학'
X['대학전공s5_대분류'][X['대학전공s5_대분류'].str.contains('법|law')] = '법과대학'

# 이과
X['대학전공s5_대분류'][X['대학전공s5_대분류'].str.contains('기계|전기|전자|화학|신소재|재료|공학|환경메카트로닉스|토목|제어계측공|Engineering|반도체|안전공|응용소재|제어 계측|세라믹|섬유|자동차|열유체|전파|조경|건설|원예|도시계발|도시개발|physics|기관공')] = '공과대학'
X['대학전공s5_대분류'][X['대학전공s5_대분류'].str.contains('수학|물리|생물|생명|지구|환경|식품|영양|생활과학|지질학|지리|식물학|원예과학|천문우주학|도시지역과학부|낙농|대기|산림자원|해양|새왈|자연과학|Chemistry|동물자원과학|임학|지적학|자연|생활자원|수족병리')] = '자연과학대학'
X['대학전공s5_대분류'][X['대학전공s5_대분류'].str.contains('컴퓨터|소프트웨어|정보|컴공|인터넷|컴정|웹서비스')] = '소프트웨어대학'

# 예체능
X['대학전공s5_대분류'][X['대학전공s5_대분류'].str.contains('미술|디자인|조소|의상|패션|섬유예술|회화|동양|서양화|서양학|의류|design|일러스트|사진|도예|공예|조형|장식|도자|시각|산업디자이|만화|fashoin|금속공|한국화|의생활|clothing and textile|애니')] = '디자인'
X['대학전공s5_대분류'][X['대학전공s5_대분류'].str.contains('작곡|영상|연극|연기|음악|국악|영화|무용|성악|피아노|music|Theatre|Music|기악|큐레이터|tesol|서양기타|더블 베이스|관현악|performance|공연|예술|연출')] = '예술대학'
X['대학전공s5_대분류'][X['대학전공s5_대분류'].str.contains('헬스|운동|레저|스포츠|건강|검도|체육|태권도|유도|체욱|트레이닝|physics|권투|발레')] = '체육대학'
X['대학전공s5_대분류'][X['대학전공s5_대분류'].str.contains('건축|도시|건설|리모델링|인테리어')] = '건축대학'
X['대학전공s5_대분류'][X['대학전공s5_대분류'].str.contains('조리|외식')] = '조리'

# 의료
X['대학전공s5_대분류'][X['대학전공s5_대분류'].str.contains('의과|의예|간호|약학|바이오|응급|구조|보건|메디컬|수의학과|청각학|의학')] = '메디컬'


In [89]:
univ_lst = ['경영학과','비즈니스', '재무회계', '어문대학', '인문대학', '사회과학대학', '미디어', '교육대학', '법과대학', '공과대학', '자연과학대학', '소프트웨어대학', '디자인', '예술대학', '체육대학', '건축대학', '조리', '메디컬']

for df in [X]:
    for i in range(len(df)):
        if df['대학전공s5_대분류'][i] not in univ_lst:
            df['대학전공s5_대분류'][i] = '기타'

In [90]:
print(len(X.대학전공s5_대분류.value_counts()))
print(X.대학전공s5_대분류.value_counts(sort=True))

19
공과대학       7473
경영학과       4576
어문대학       3317
사회과학대학     2480
디자인        2206
재무회계       1658
자연과학대학     1280
인문대학       1191
소프트웨어대학     944
미디어         630
법과대학        519
체육대학        262
비즈니스        252
교육대학        251
기타          243
예술대학        137
건축대학         99
메디컬          89
조리           11
Name: 대학전공s5_대분류, dtype: int64


# 대학전공_소분류

In [91]:
X['대학전공s5_소분류'] = X['대학전공'].copy()

#### 인문계열

In [92]:
a = X[X['대학전공s5_소분류'].str.contains('국어|국문|한국어')]['대학전공s5_소분류']
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'국어국문학과')
a = X[X['대학전공s5_소분류'].str.contains('중어|중문|중국|한문|Chinese|중국어')]['대학전공s5_소분류']
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'중문학과')
a = X[X['대학전공s5_소분류'].str.contains('불어|프랑스어')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'불어학과')
a = X[X['대학전공s5_소분류'].str.contains('영어|영문|영미|어학연수|English')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'영문학과')
a = X[X['대학전공s5_소분류'].str.contains('일어|일본어')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'일본어학과')
a = X[X['대학전공s5_소분류'].str.contains('스페인어')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'스페인어학과')
a = X[X['대학전공s5_소분류'].str.contains('독어|독문|독일어')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'독어독문학과')
a = X[X['대학전공s5_소분류'].str.contains('러시아|러시아어')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'러시아어학과')
a = X[X['대학전공s5_소분류'].str.contains('고고학|국사|사학|역사')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'역사학과')
a = X[X['대학전공s5_소분류'].str.contains('한문|한자')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'한문학과')
a = X[X['대학전공s5_소분류'].str.contains('인문')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'인문학과')
a = X[X['대학전공s5_소분류'].str.contains('철학|윤리')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'철학과')
a = X[X['대학전공s5_소분류'].str.contains('심리')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'심리학과')
a = X[X['대학전공s5_소분류'].str.contains('문예|창작')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'문예창작과')
a = X[X['대학전공s5_소분류'].str.contains('문헌정보|문헌')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'문헌학과')
a = X[X['대학전공s5_소분류'].str.contains('문화|미술사학|민속')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'민속학과')

#### 사회계열

In [93]:
a = X[X['대학전공s5_소분류'].str.contains('관광|호텔')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'호텔관광학과')
a = X[X['대학전공s5_소분류'].str.contains('행정|핼정학|Public Administration')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'행정학과')
a = X[X['대학전공s5_소분류'].str.contains('신문|방송')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'신문방송학과')
a = X[X['대학전공s5_소분류'].str.contains('언론')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'언론학과')
a = X[X['대학전공s5_소분류'].str.contains('미디어|정보미디어|콘텐츠')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'미디어학과')
a = X[X['대학전공s5_소분류'].str.contains('법학|법률')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'법학과')
a = X[X['대학전공s5_소분류'].str.contains('경제|경상')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'경제학과')
a = X[X['대학전공s5_소분류'].str.contains('경영|MANAGEMENT|경역')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'경영학과')
a = X[X['대학전공s5_소분류'].str.contains('통상|무역|물류|Trade')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'무역학과')
a = X[X['대학전공s5_소분류'].str.contains('사회')]['대학전공s5_소분류']
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'사회학과')
a = X[X['대학전공s5_소분류'].str.contains('재무')]['대학전공s5_소분류']
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'재무학과')
a = X[X['대학전공s5_소분류'].str.contains('회계|세무')]['대학전공s5_소분류']
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'회계학과')
a = X[X['대학전공s5_소분류'].str.contains('정치|외교|정외')]['대학전공s5_소분류']
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'정치학과')
a = X[X['대학전공s5_소분류'].str.contains('경찰')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'경찰학과')
a = X[X['대학전공s5_소분류'].str.contains('광고|홍보|광보홍보')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'광고홍보학과')
a = X[X['대학전공s5_소분류'].str.contains('국제')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'국제학과')
a = X[X['대학전공s5_소분류'].str.contains('노인|복지')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'노인복지학과')
a = X[X['대학전공s5_소분류'].str.contains('비서')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'비서학과')
a = X[X['대학전공s5_소분류'].str.contains('사회복지')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'사회복지학과')
a = X[X['대학전공s5_소분류'].str.contains('사회')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'사회')
a = X[X['대학전공s5_소분류'].str.contains('지리|지리학|지적')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'지리학과')
a = X[X['대학전공s5_소분류'].str.contains('항공','항공서비스')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'항공학과')

#### 교육계열

In [94]:
a = X[X['대학전공s5_소분류'].str.contains('교육|사범')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'교육학과')

#### 자연계열

In [95]:
a = X[X['대학전공s5_소분류'].str.contains('가정관리|가정')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'가정관리학과')
a = X[X['대학전공s5_소분류'].str.contains('농업|농과대|농생물|농업학|농학')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'농업학과')
a = X[X['대학전공s5_소분류'].str.contains('물리|physics')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'물리학과')
a = X[X['대학전공s5_소분류'].str.contains('산림|원예')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'산림학과')
a = X[X['대학전공s5_소분류'].str.contains('생명')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'생명학과')
a = X[X['대학전공s5_소분류'].str.contains('수산')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'수산학과')
a = X[X['대학전공s5_소분류'].str.contains('식품|영양')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'식품영양')
a = X[X['대학전공s5_소분류'].str.contains('조리')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'조리학과')
a = X[X['대학전공s5_소분류'].str.contains('지구|지구과학|지질')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'지구과학과')
a = X[X['대학전공s5_소분류'].str.contains('천문|기상')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'천문학과')
a = X[X['대학전공s5_소분류'].str.contains('통계')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'통계학과')
a = X[X['대학전공s5_소분류'].str.contains('화학|화공|화확|Chemistry|회힉')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'화학과')

## 공학계열

In [96]:
a = X[X['대학전공s5_소분류'].str.contains('정보통신')]['대학전공s5_소분류']
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'정보통신공학과')
a = X[X['대학전공s5_소분류'].str.contains('컴퓨터|컴공|켬퓨터|컴정|서버|네트워크')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'컴퓨터공학과')
a = X[X['대학전공s5_소분류'].str.contains('소프트웨어|웹')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'소프트웨어공학과')
a = X[X['대학전공s5_소분류'].str.contains('신소재|신재생')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'신소재공학과')
a = X[X['대학전공s5_소분류'].str.contains('건축|설비')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'건축학과')
a = X[X['대학전공s5_소분류'].str.contains('게임')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'게임학과')
a = X[X['대학전공s5_소분류'].str.contains('기계')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'기계학과')
a = X[X['대학전공s5_소분류'].str.contains('메카트로')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'메카트로닉스학과')
a = X[X['대학전공s5_소분류'].str.contains('반도체|세라믹')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'반도체학과')
a = X[X['대학전공s5_소분류'].str.contains('산업공학')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'산업공학과')
a = X[X['대학전공s5_소분류'].str.contains('섬유')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'섬유학과')
a = X[X['대학전공s5_소분류'].str.contains('광학|안경')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'안경학과')
a = X[X['대학전공s5_소분류'].str.contains('에너지')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'에너지학과')
a = X[X['대학전공s5_소분류'].str.contains('자동차')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'자동차학과')
a = X[X['대학전공s5_소분류'].str.contains('재료|금속')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'재료공학과')
a = X[X['대학전공s5_소분류'].str.contains('전기')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'전기학과')
a = X[X['대학전공s5_소분류'].str.contains('전자')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'전자학과')
a = X[X['대학전공s5_소분류'].str.contains('정보보안|보호')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'정보보안학과')
a = X[X['대학전공s5_소분류'].str.contains('제어')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'제어학과')
a = X[X['대학전공s5_소분류'].str.contains('조경')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'조경학과')
a = X[X['대학전공s5_소분류'].str.contains('조경')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'조경학과')
a = X[X['대학전공s5_소분류'].str.contains('교통')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'교통공학과')
a = X[X['대학전공s5_소분류'].str.contains('토목')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'토목학과')
a = X[X['대학전공s5_소분류'].str.contains('해양')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'해양공학과')
a = X[X['대학전공s5_소분류'].str.contains('환경|환격')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'환경공학과')
a = X[X['대학전공s5_소분류'].str.contains('도시')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'도시공학과')

#### 예체능 계열

In [97]:
a = X[X['대학전공s5_소분류'].str.contains('경호')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'경호학과')
a = X[X['대학전공s5_소분류'].str.contains('공예')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'공예학과')
a = X[X['대학전공s5_소분류'].str.contains('만화|애니메이션')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'애니메이션학과')
a = X[X['대학전공s5_소분류'].str.contains('무용')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'무용학과')
a = X[X['대학전공s5_소분류'].str.contains('미술|한국화|회화')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'미술학과')
a = X[X['대학전공s5_소분류'].str.contains('방송연예')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'방송연예학과')
a = X[X['대학전공s5_소분류'].str.contains('사진|영상')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'사진영상학과')
a = X[X['대학전공s5_소분류'].str.contains('음악|관현악')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'음악학과')
a = X[X['대학전공s5_소분류'].str.contains('시각')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'시각학과')
a = X[X['대학전공s5_소분류'].str.contains('연극|영화')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'연극영화학과')
a = X[X['대학전공s5_소분류'].str.contains('체육|체욱|태권도')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'체육학과')
a = X[X['대학전공s5_소분류'].str.contains('조형')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'조형학과')
a = X[X['대학전공s5_소분류'].str.contains('패션|의류|의상')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'패션디자인학과')

#### 의학계열

In [98]:
a = X[X['대학전공s5_소분류'].str.contains('간호')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'간호학과')
a = X[X['대학전공s5_소분류'].str.contains('보건')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'보건학과')
a = X[X['대학전공s5_소분류'].str.contains('약학')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'약학과')
a = X[X['대학전공s5_소분류'].str.contains('응급구조|구조')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'응급구조학과')
a = X[X['대학전공s5_소분류'].str.contains('의학|의과대학|의생활|의예|의학과')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'의학과')
a = X[X['대학전공s5_소분류'].str.contains('임상병리')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'임상병리학과')

#### 세부사항 변경

In [99]:
a = X[X['대학전공s5_소분류'].str.contains('수학')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'수학과')
a = X[X['대학전공s5_소분류'].str.contains('이비즈니스|BUSINESS|비즈니스|e-|비지니스')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'비즈니스과')
a = X[X['대학전공s5_소분류'].str.contains('커뮤니케이션|Communication')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'커뮤니케이션')
a = X[X['대학전공s5_소분류'].str.contains('Management')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'관리학과')
a = X[X['대학전공s5_소분류'].str.contains('음악|Music|피아노')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'음악학과')
a = X[X['대학전공s5_소분류'].str.contains('가구')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'가구학과')
a = X[X['대학전공s5_소분류'].str.contains('건강')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'건강학과')
a = X[X['대학전공s5_소분류'].str.contains('건설공|건설시|건축|건충')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'건설학과')
a = X[X['대학전공s5_소분류'].str.contains('격기')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'격기학과')
a = X[X['대학전공s5_소분류'].str.contains('경기')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'경기지도학과')
a = X[X['대학전공s5_소분류'].str.contains('인류')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'고고인류학과')
a = X[X['대학전공s5_소분류'].str.contains('고분자')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'고분자학과')
a = X[X['대학전공s5_소분류'].str.contains('공간')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'공간학과')
a = X[X['대학전공s5_소분류'].str.contains('공업')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'공업학과')
a = X[X['대학전공s5_소분류'].str.contains('관경|관공통역|관좡')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'관광학과')
a = X[X['대학전공s5_소분류'].str.contains('통신공학|광통신')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'통신학과')
a = X[X['대학전공s5_소분류'].str.contains('교정학')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'교육학과')
a = X[X['대학전공s5_소분류'].str.contains('국악')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'국악과')
a = X[X['대학전공s5_소분류'].str.contains('종합|선물보헙|선물')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'금융학과')
a = X[X['대학전공s5_소분류'].str.contains('기계|기게공')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'기계공학과')
a = X[X['대학전공s5_소분류'].str.contains('기공|기관공')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'기관공학과')
a = X[X['대학전공s5_소분류'].str.contains('기독교')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'기독교학과')
a = X[X['대학전공s5_소분류'].str.contains('기악')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'기악학과')
a = X[X['대학전공s5_소분류'].str.contains('축산|낙농')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'낙농학과')
a = X[X['대학전공s5_소분류'].str.contains('노문학|노어')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'노어학과')
a = X[X['대학전공s5_소분류'].str.contains('신문방송학과|언론학|다중매체')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'매체학과')
a = X[X['대학전공s5_소분류'].str.contains('데이터')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'데이터학과')
a = X[X['대학전공s5_소분류'].str.contains('도서관|library')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'도서관학과')
a = X[X['대학전공s5_소분류'].str.contains('도예|도자')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'도예학과')
a = X[X['대학전공s5_소분류'].str.contains('독어독|독일')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'독일학과')
a = X[X['대학전공s5_소분류'].str.contains('동물자원|동물과학')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'동물학과')
a = X[X['대학전공s5_소분류'].str.contains('동양무')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'동양무예학과')
a = X[X['대학전공s5_소분류'].str.contains('동양어문')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'동양학과')
a = X[X['대학전공s5_소분류'].str.contains('동양화')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'동양화')
a = X[X['대학전공s5_소분류'].str.contains('디지털')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'디지털학과')
a = X[X['대학전공s5_소분류'].str.contains('레저|레져')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'레저학과')
a = X[X['대학전공s5_소분류'].str.contains('마케팅')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'마케팅학과')
a = X[X['대학전공s5_소분류'].str.contains('목회')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'목회학과')
a = X[X['대학전공s5_소분류'].str.contains('무대디자인')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'무대디자인학과')
a = X[X['대학전공s5_소분류'].str.contains('미생물')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'미생물학과')
a = X[X['대학전공s5_소분류'].str.contains('법|law')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'법학과')
a = X[X['대학전공s5_소분류'].str.contains('보험')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'보험학과')
a = X[X['대학전공s5_소분류'].str.contains('부동산')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'부동산학과')
a = X[X['대학전공s5_소분류'].str.contains('분자생물')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'분자생물학과')
a = X[X['대학전공s5_소분류'].str.contains('불문|불어')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'불어학과')
a = X[X['대학전공s5_소분류'].str.contains('사료')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'사료학과')
a = X[X['대학전공s5_소분류'].str.contains('사하|사회')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'사회학과')
a = X[X['대학전공s5_소분류'].str.contains('산업공|산업도예|산업시|산업안|산업재|산업정보')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'산업학과')
a = X[X['대학전공s5_소분류'].str.contains('산업 디자인|산업디자|산업제품디자인')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'산업디자인학과')  
a = X[X['대학전공s5_소분류'].str.contains('상경')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'상경학과')
a = X[X['대학전공s5_소분류'].str.contains('생명|생물|생물자원')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'생물학과')
a = X[X['대학전공s5_소분류'].str.contains('생산')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'생산학과')
a = X[X['대학전공s5_소분류'].str.contains('생활과|생활자원')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'생활학과')
a = X[X['대학전공s5_소분류'].str.contains('서양사|서양학과')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'서양학과')
a = X[X['대학전공s5_소분류'].str.contains('서양화')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'서양화')
a = X[X['대학전공s5_소분류'].str.contains('선교')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'선교학과')
a = X[X['대학전공s5_소분류'].str.contains('성악')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'성악학과')
a = X[X['대학전공s5_소분류'].str.contains('소비자 인간|소비자가족|소비자아동')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'소비자학과')
a = X[X['대학전공s5_소분류'].str.contains('소트웨어|소프트웨어')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'소프트웨어학과')
a = X[X['대학전공s5_소분류'].str.contains('수족병리')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'수족병리학과')
a = X[X['대학전공s5_소분류'].str.contains('스포츠')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'스포츠학과')
a = X[X['대학전공s5_소분류'].str.contains('식량자원공학|식량자원학과')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'식량자원학과')
a = X[X['대학전공s5_소분류'].str.contains('신학')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'신학과')
a = X[X['대학전공s5_소분류'].str.contains('아동|유아')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'아동학과')
a = X[X['대학전공s5_소분류'].str.contains('안전')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'안전학과')
a = X[X['대학전공s5_소분류'].str.contains('애니')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'애니학과')
a = X[X['대학전공s5_소분류'].str.contains('양식')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'양식학과')
a = X[X['대학전공s5_소분류'].str.contains('연극영화|연기|Theatre')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'연기학과')
a = X[X['대학전공s5_소분류'].str.contains('예술대학|예술학부|체능')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'예술학과')
a = X[X['대학전공s5_소분류'].str.contains('요업공학')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'요업공학과')
a = X[X['대학전공s5_소분류'].str.contains('운동기능|웨이트트레이닝|헬스')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'운동학과')
a = X[X['대학전공s5_소분류'].str.contains('유도')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'유도학과')
a = X[X['대학전공s5_소분류'].str.contains('유전공')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'유전공학과')
a = X[X['대학전공s5_소분류'].str.contains('유통')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'유통학과')
a = X[X['대학전공s5_소분류'].str.contains('유학과|유학동양')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'유학')
a = X[X['대학전공s5_소분류'].str.contains('양식')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'양식학과')
a = X[X['대학전공s5_소분류'].str.contains('소재')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'소재학과')
a = X[X['대학전공s5_소분류'].str.contains('인터넷')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'인터넷학과')
a = X[X['대학전공s5_소분류'].str.contains('인테리어')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'인테리어학과')
a = X[X['대학전공s5_소분류'].str.contains('일본|일문')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'일본학과')
a = X[X['대학전공s5_소분류'].str.contains('임상|임학')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'임상학과')
a = X[X['대학전공s5_소분류'].str.contains('양식')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'양식학과')
a = X[X['대학전공s5_소분류'].str.contains('자원')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'자원학과')
a = X[X['대학전공s5_소분류'].str.contains('작곡')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'작곡학과')
a = X[X['대학전공s5_소분류'].str.contains('전산')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'전산학과')
a = X[X['대학전공s5_소분류'].str.contains('전파|전자')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'전파학과')
a = X[X['대학전공s5_소분류'].str.contains('정보')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'정보학과')
a = X[X['대학전공s5_소분류'].str.contains('제약')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'제약학과')
a = X[X['대학전공s5_소분류'].str.contains('조소')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'조소학과')
a = X[X['대학전공s5_소분류'].str.contains('주거솬|주생활')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'주거학과')
a = X[X['대학전공s5_소분류'].str.contains('양식')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'양식학과')
a = X[X['대학전공s5_소분류'].str.contains('지식정보')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'지식정보학과')
a = X[X['대학전공s5_소분류'].str.contains('지역개발|지역계획|지역정보')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'지역학과')
a = X[X['대학전공s5_소분류'].str.contains('청소년')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'청소년학과')
a = X[X['대학전공s5_소분류'].str.contains('양식')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'축산학과')
a = X[X['대학전공s5_소분류'].str.contains('토&#4|토목학|토옥')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'토목공과')
a = X[X['대학전공s5_소분류'].str.contains('수원대|&#49887;품공학|1|2급|3.|F-H|Summer|Material|SOC|clothing|ddd|ghj|tesol|ㅇ|없음|이공계|졸업|준사서|증식|철철|해선|eh|경매|더블|상학과|마이컴|Social|광보')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'기타')
a = X[X['대학전공s5_소분류'].str.contains('복식디자인|비주얼|생활디자인|서피스|스페이스|실내디자인|실내장식|써피스|fashoin|일러스트|장신구|제품디자인|텍스타일|패션|편집|지자인')]['대학전공s5_소분류'] 
X['대학전공s5_소분류'] = X['대학전공s5_소분류'].replace(list(a),'디자인학과')

In [100]:
univ_lst = ['국어국문학과', '안경학과', '신학과', '화학과', '컴퓨터공학과', '역사학과', '일본학과', '영문학과',
       '불어학과', '산업학과', '통계학과', '매체학과', '미술학과', '문예창작과', '무역학과', '행정학과',
       '경영학과', '유통학과', '환경공학과', '기계공학과', '경제학과', '물리학과', '전파학과', '정치학과',
       '건설학과', '전기학과', '전산학과', '수학과', '디자인학과', '법학과', '재료공학과', '유학',
       '생물학과', '교육학과', '회계학과', '사회학과', '보험학과', '광고홍보학과', '토목공과', '간호학과',
       '중문학과', '가정관리학과', '자원학과', '식품영양', '소프트웨어학과', '음악학과', '시각학과',
       '도시공학과', '유전공학과', '성악학과', '산업디자인학과', '제어학과', '대기과학', '산림학과',
       '보건학과', '미디어학과', '철학과', '심리학과', '데이터학과', '독일학과', '낙농학과', '문헌학과',
       '메카트로닉스학과', '정보학과', '동양학과', '공예학과', '동물학과', '소재학과', '교통공학과', '서양화',
       '체육학과', '무용학과', '무대디자인학과', '노인복지학과', '동양화', '기타', '조형학과',
       '부동산학과', '재무학과', '공학계열', '아동학과', '지구과학과', '도서관학과', '지역학과', '민속학과',
       '스포츠학과', '러시아어학과', '비서학과', '요업공학과', '고분자학과', '수산학과', '비즈니스과',
       '조경학과', '약학과', '선교학과', '애니학과', '상업공학과', '기독교학과', '도예학과',
       '농업학과', '안전학과', '격기학과', '조소학과', '청소년학과', '의학과', '해양공학과', '어문학부',
       '작곡학과', '기악학과', '축산학과', '국제학과', '공업학과', '전공', '유도학과', '항공학과',
       '지리학과', '레저학과', 'MIS', '마케팅학과', '노어학과', '그래픽디자인', '경기지도학과',
       '소비자학', '국악과', '금융학과', '식물학', '고고인류학과', '반도체학과', '기관공학과', '생산학과',
       '수족병리학과', '디자인과', '연기학과', '목회학과', '생활학과', '스페인어학과', '소비자인간발달',
       '예술학과', '금융', '건강학과', '검도', '소비자학과', '조리학과', '디자인', '금융학',
       '제약학과', '외식사업', '노년학', '관리학과', '인터넷학과', '운동학과', '자연과학부', '사료학과',
       '주거학과', '경호학과', '자동차학과', '보육학과', '언어학과', '인문학과', '에너지학과', '가구학과',
       '설계전공', '상경학과', '소비자인간발달학과', '인테리어학과', '디자인학부', '경찰학과', '디자인대학교',
       '리모델링', '공학', '열유체', '한국학', '천문학과', '디지털학과', '동양무예학과',
       '서양학과', '큐레이터학', '바이오케미컬', '매체공학', '응급구조학과', '북한학과']

for df in [X]:
    for i in range(len(df)):
        if df['대학전공s5_소분류'][i] not in univ_lst:
            df['대학전공s5_소분류'][i] = '기타'

In [101]:
X= X.copy()
X['대학전공'] = X['대학전공'].str[:2]
X['대학전공'].value_counts()

경영    2877
전자    1434
컴퓨    1293
산업    1252
기계    1054
      ... 
To       1
ㅇㅇ       1
우통       1
소트       1
융합       1
Name: 대학전공, Length: 414, dtype: int64

### 출신대학_지역(인서울, 경기권, 지방권)

In [102]:
X['출신대학s5_지역'] = X['출신대학'].copy()

In [103]:
X['출신대학s5_지역'][X['출신대학s5_지역'].str.contains('성균관대학교|세종대학교|성신여자대학교|연세대학교|동덕여자대학교|한성대학교|성공회대학교|중앙대학교|이화여자대학교|서울여자대학교|서울과학기술대학교')] = '서울'

X['출신대학s5_지역'][X['출신대학s5_지역'].str.contains('수원대학교|인천대학교|경기대학교|성결대학교|용인대학교|협성대학교|한국산업기술대학교')] = '경기'

X['출신대학s5_지역'][X['출신대학s5_지역'].str.contains('신라대학교|한밭대학교|전주대학교|제주대학교|세명대학교|상지대학교|선문대학교|호원대학교|한림대학교|목포대학교|동아대학교|군산대학교|동의대학교|호남대학교|광주대학교|서원대학교|동서대학교|한일장신대학교|한세대학교|경주대학교|서울신학대학교|송원대학교|부산디지털대학교')] = '지방'

### 근무경력 데이터 타입 변경

In [104]:
# 근무경력 dtype object -> int
X = X.astype({ '근무경력' : 'int' })

### 직무태그 앞에 두 글자만 뽑아내기

In [105]:
# 직무태그 열 복사해서 앞 두 글자만 뽑아내기

X['직무태그_1'] = X['직무태그'].str[:2]

# 직무태그 결측값 채우기
X.직무태그_1 = X.직무태그_1.fillna('없음')

## 근무 지역 ➡ 근무지역_소분류, 근무지역_대분류 feature 생성

### 근무지역_소분류

In [106]:
X['근무지역_소분류'] = X['근무지역'].copy()

In [107]:
X['근무지역_소분류'][X['근무지역_소분류'].str.contains
                ('해외|미국|중국|러시아|일본|캐나다|필리핀|인도|싱가포르|대만|말레이시아|프랑스|홍콩')] = '해외'

X['근무지역_소분류'][X['근무지역_소분류'] == '전국,,'] = '전국'
X['근무지역_소분류'][X['근무지역_소분류'] == '전국,전국,'] = '전국'
X['근무지역_소분류'][X['근무지역_소분류'] == '전국,전국,전국'] = '전국'

X['근무지역_소분류'][X['근무지역_소분류'].str.split(',').str[0] == '서울'] = '서울'
X['근무지역_소분류'][X['근무지역_소분류'].str.split(',').str[0] == '경기'] = '경기'
X['근무지역_소분류'][X['근무지역_소분류'].str.split(',').str[0] == '인천'] = '인천'
X['근무지역_소분류'][X['근무지역_소분류'].str.split(',').str[0] == '부산'] = '부산'
X['근무지역_소분류'][X['근무지역_소분류'].str.split(',').str[0] == '대전'] = '대전'
X['근무지역_소분류'][X['근무지역_소분류'].str.split(',').str[0] == '광주'] = '광주'
X['근무지역_소분류'][X['근무지역_소분류'].str.split(',').str[0] == '충남'] = '충남'
X['근무지역_소분류'][X['근무지역_소분류'].str.split(',').str[0] == '경남'] = '경남'
X['근무지역_소분류'][X['근무지역_소분류'].str.split(',').str[0] == '전북'] = '전북'
X['근무지역_소분류'][X['근무지역_소분류'].str.split(',').str[0] == '충북'] = '충북'
X['근무지역_소분류'][X['근무지역_소분류'].str.split(',').str[0] == '제주'] = '제주'
X['근무지역_소분류'][X['근무지역_소분류'].str.split(',').str[0] == '경북'] = '경북'
X['근무지역_소분류'][X['근무지역_소분류'].str.split(',').str[0] == '강원'] = '강원'
X['근무지역_소분류'][X['근무지역_소분류'].str.split(',').str[0] == '울산'] = '울산'
X['근무지역_소분류'][X['근무지역_소분류'].str.split(',').str[0] == '전남'] = '전남'
X['근무지역_소분류'][X['근무지역_소분류'].str.split(',').str[0] == '대구'] = '대구'
X['근무지역_소분류'][X['근무지역_소분류'].str.split(',').str[0] == '전북'] = '전북'

X['근무지역_소분류'][X['근무지역_소분류'].str.contains
                ('기타')] = '기타'

### 근무지역_대분류

In [108]:
X['근무지역_대분류'] = X['근무지역_소분류'].copy()

In [109]:
X['근무지역_대분류'][X['근무지역_대분류'].str.contains('서울|인천|경기')] = '수도권'
X['근무지역_대분류'][~X['근무지역_대분류'].str.contains('수도권|해외|전국')] = '비수도권'

In [115]:
X['직무태그_1'].value_counts()

없음    4213
영업     988
마케     714
경영     670
무역     642
      ... 
별정       1
통번       1
응요       1
이제       1
해양       1
Name: 직무태그_1, Length: 1484, dtype: int64

In [116]:
def resumetable(df):
    print(f'데이터셋 형상: {df.shape}')
    summary = pd.DataFrame(df.dtypes, columns=['데이터 타입'])
    summary = summary.reset_index()
    summary = summary.rename(columns={'index': '피처'})
    summary['결측값 개수'] = df.isnull().sum().values
    summary['고윳값 개수'] = df.nunique().values
    summary['첫 번째 값'] = df.loc[0].values
    summary['두 번째 값'] = df.loc[1].values
    summary['세 번째 값'] = df.loc[2].values
    
    return summary

feature_resume_table = resumetable(X)
feature_resume_table

데이터셋 형상: (27618, 17)


,피처,데이터 타입,결측값 개수,고윳값 개수,첫 번째 값,두 번째 값,세 번째 값
0,직종,object,0,12,문화·예술·신문·방송,경영·기획·회계·사무,IT·게임
1,세부직종,object,0,115,영상·음향·사진·카메라,사무·총무·법무,하드웨어설계·개발·관리
2,직무태그,object,4209,19093,"취재기자, 편집 기사, 유아 사이트 기획, 시나리오 작성",NaN,"하드웨어 , 무선통신 , MICOM , ASM , RF , CDMA , Firm W..."
3,근무경력,int64,0,338,0,35,0
4,근무형태,object,0,31,비정규직,정규직,비정규직
5,근무지역,object,0,1690,"서울,경기,서울","부산,서울,일본","서울,경기,"
6,출신대학,object,0,42,성균관대학교,신라대학교,수원대학교
7,대학전공,object,0,414,국문,관광,정보
8,어학시험,object,0,29,없음,JLPT,없음
9,자격증,int64,0,2,0,0,1


In [117]:
X

,직종,세부직종,직무태그,근무경력,근무형태,근무지역,출신대학,대학전공,어학시험,자격증,대학성적,대학전공s5_대분류,대학전공s5_소분류,출신대학s5_지역,직무태그_1,근무지역_소분류,근무지역_대분류
0,문화·예술·신문·방송,영상·음향·사진·카메라,"취재기자, 편집 기사, 유아 사이트 기획, 시나리오 작성",0,비정규직,"서울,경기,서울",성균관대학교,국문,없음,0,70.0,어문대학,국어국문학과,서울,취재,서울,수도권
1,경영·기획·회계·사무,사무·총무·법무,NaN,35,정규직,"부산,서울,일본",신라대학교,관광,JLPT,0,70.0,경영학과,안경학과,지방,없음,해외,해외
2,IT·게임,하드웨어설계·개발·관리,"하드웨어 , 무선통신 , MICOM , ASM , RF , CDMA , Firm W...",0,비정규직,"서울,경기,",수원대학교,정보,없음,1,60.0,소프트웨어대학,신학과,경기,하드,서울,수도권
3,영업·판매·TM,기술영업,반도체,12,정규직,"경기,서울,충북",수원대학교,정보,없음,1,70.0,소프트웨어대학,신학과,경기,반도,경기,수도권
4,기술·과학·산업,기타 기술·과학·산업,기술직,0,비정규직,"전국,전국,전국",한밭대학교,화학,없음,0,70.0,공과대학,화학과,지방,기술,전국,전국
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27613,디자인,제품·산업디자인,"제품,산업,팬시",0,비정규직,"서울,부산,",호원대학교,섬유,없음,0,70.0,공과대학,유학,지방,제품,서울,수도권
27614,경영·기획·회계·사무,물류·유통관리,"물류,유통",0,비정규직,"경기,인천,서울",협성대학교,유통,없음,0,70.0,경영학과,경영학과,경기,물류,경기,수도권
27615,재료·화학·섬유·의복,섬유·의복,NaN,0,계약직,"서울,경기,",한성대학교,의류,없음,0,70.0,디자인,디자인학과,서울,없음,서울,수도권
27616,영업·판매·TM,판매·매장관리·캐셔·프런트,판매,0,비정규직,"부산,부산,부산",동서대학교,식품,TOEIC,0,70.0,공과대학,식품영양,지방,판매,부산,비수도권


In [118]:
X['직종'].value_counts()

경영·기획·회계·사무    12197
IT·게임           4019
영업·판매·TM        3879
재료·화학·섬유·의복     2085
건설·기계·전기·전자     2011
전문·교육·자격        1499
디자인             1019
통신·모바일           302
기술·과학·산업         243
문화·예술·신문·방송      200
기타 직종            163
호텔/콘도/리조트          1
Name: 직종, dtype: int64

In [119]:
# 직무태그 열 복사해서 앞 두 글자만 뽑아내기

X['세부직종']

0          영상·음향·사진·카메라
1              사무·총무·법무
2          하드웨어설계·개발·관리
3                  기술영업
4           기타 기술·과학·산업
              ...      
27613          제품·산업디자인
27614           물류·유통관리
27615             섬유·의복
27616    판매·매장관리·캐셔·프런트
27617          호텔·숙박·여행
Name: 세부직종, Length: 27618, dtype: object

# 직무태그 글자수 범위

In [120]:
X['직무태그_2'] = X['직무태그'].copy()

In [121]:
#직무태그 컬럼에서 결측치 없으면 직무태그 컬럼 값 그대로 사용, 결측치가 있을경우, 세부직종 컬럼으로 결측치 채움

X['직무태그_2']= np.where(pd.notnull(X['직무태그_2'])== True, X['직무태그_2'], X['세부직종'])

In [122]:
a = []
for i in X['직무태그_2']:
        a.append(len(i))
        
X['직무태그글자수'] = a

In [123]:
X['직무태그_2']

0                          취재기자, 편집 기사, 유아 사이트 기획, 시나리오 작성
1                                                 사무·총무·법무
2        하드웨어 , 무선통신 , MICOM , ASM , RF , CDMA , Firm W...
3                                                      반도체
4                                                      기술직
                               ...                        
27613                                             제품,산업,팬시
27614                                                물류,유통
27615                                                섬유·의복
27616                                                   판매
27617                                             호텔·숙박·여행
Name: 직무태그_2, Length: 27618, dtype: object

In [124]:
a = []
for i in X['직무태그글자수']:
    if 10 > i >= 0:
        a.append(10)
    elif 20 > i >= 10:
        a.append(20)
    elif 30 > i >= 20:
        a.append(30)
    elif 40 > i >= 30:
        a.append(40)
    elif 50 > i >= 40:
        a.append(50)
    elif 60 > i >= 50:
        a.append(60)
    elif 70 > i >= 60:
        a.append(70)
    else:
        a.append(80)

X['직무태그글자수범위별'] = a
X = X.drop(columns = '직무태그글자수')
X = X.drop(columns = '직무태그_2')

In [125]:
X.columns

Index(['직종', '세부직종', '직무태그', '근무경력', '근무형태', '근무지역', '출신대학', '대학전공', '어학시험',
       '자격증', '대학성적', '대학전공s5_대분류', '대학전공s5_소분류', '출신대학s5_지역', '직무태그_1',
       '근무지역_소분류', '근무지역_대분류', '직무태그글자수범위별'],
      dtype='object')

## 대학성적 점수대별 

# (1) Ratio Features

In [126]:
# f = X.groupby('')[ ]


# in_seoul = X[X['출신대학s5_지역'] == '서울'].value_counts().index[1:].to_list()
# in_gyeonggi = X[X['출신대학s5_지역'] == '경기'].value_counts().index[1:].to_list()
# in_jibang = X[X['출신대학s5_지역'] == '지방'].value_counts().index[1:].to_list()

# (2) Aggregation Features

Category와 numerical feature의 조합으로 생성함.

Category 각 그룹당 mean, median, variance, standard deviation을 feature로 사용

### 1. 근무형태 + 근무경력

In [127]:
object1 = X.groupby('근무형태')['근무경력'].agg([('근무형태_총근무경력s5', np.sum),
                                           #  ('구매건수', np.size),
                                            ('근무형태_평균근무경력s5', np.mean),
                                            ('근무형태_최대근무경력s5', np.max),
                                            ('근무형태_최소근무경력s5', np.min),
                                            ('근무형태_근무경력표준편차s5', np.std),   
                                            ('근무형태_근무경력변동계수s5', lambda x : np.std(x)/np.mean(x))]).reset_index().fillna(0)

X = pd.merge(X, object1, on = '근무형태', how='left')

### 2. 근무형태 + 대학성적

In [128]:
object3 = X.groupby('근무형태')['대학성적'].agg([('근무형태_총대학성적s5', np.sum),
                                           #  ('구매건수', np.size),
                                            ('근무형태_평균대학성적s5', np.mean),
                                            ('근무형태_최대대학성적s5', np.max),
                                            ('근무형태_최소대학성적s5', np.min),
                                            ('근무형태_대학성적표준편차s5', np.std),   
                                            ('근무형태_대학성적변동계수s5', lambda x : np.std(x)/np.mean(x))]).reset_index().fillna(0)


X = pd.merge(X, object3, on = '근무형태', how='left')

### 3. 출신대학 + 근무경력

In [129]:
object4 = X.groupby('출신대학')['근무경력'].agg([('출신대학_총근무경력s5', np.sum),
                                           #  ('구매건수', np.size),
                                            ('출신대학_평균근무경력s5', np.mean),
                                            ('출신대학_최대근무경력s5', np.max),
                                            ('출신대학_최소근무경력s5', np.min),
                                            ('출신대학_근무경력표준편차s5', np.std),   
                                            ('출신대학_근무경력변동계수s5', lambda x : np.std(x)/np.mean(x))]).reset_index().fillna(0)


X= pd.merge(X, object4, on = '출신대학', how='left')

### 4. 출신대학 + 대학성적

In [130]:
object2 = X.groupby('출신대학')['대학성적'].agg([('출신대학_총대학성적s5', np.sum),
                                           #  ('구매건수', np.size),
                                            ('출신대학_평균대학성적s5', np.mean),
                                            ('출신대학_최대대학성적s5', np.max),
                                            ('출신대학_최소대학성적s5', np.min),
                                            ('출신대학_대학성적표준편차s5', np.std),   
                                            ('출신대학_대학성적변동계수s5', lambda x : np.std(x)/np.mean(x))]).reset_index().fillna(0)


X = pd.merge(X, object2, on = '출신대학', how='left')

### 5. 출신대학_지역 + 근무경력

In [131]:
object5 = X.groupby('출신대학s5_지역')['근무경력'].agg([('출신대학_지역_총근무경력s5', np.sum),
                                           #  ('구매건수', np.size),
                                            ('출신대학s5_지역_평균근무경력s5', np.mean),
                                            ('출신대학s5_지역_최대근무경력s5', np.max),
                                            ('출신대학s5_지역_최소근무경력s5', np.min),
                                            ('출신대학s5_지역_근무경력표준편차s5', np.std),   
                                            ('출신대학s5_지역_근무경력변동계수s5', lambda x : np.std(x)/np.mean(x))]).reset_index().fillna(0)


X= pd.merge(X, object5, on = '출신대학s5_지역', how='left')

### 6. 출신대학_지역 + 대학성적

In [132]:
object60 = X.groupby('출신대학s5_지역')['대학성적'].agg([('출신대학_지역_총대학성적s5', np.sum),
                                           #  ('구매건수', np.size),
                                            ('출신대학s5_지역_평균대학성적s5', np.mean),
                                            ('출신대학s5_지역_최대대학성적s5', np.max),
                                            ('출신대학s5_지역_최소대학성적s5', np.min),
                                            ('출신대학s5_지역_대학성적표준편차s5', np.std),   
                                            ('출신대학s5_지역_대학성적변동계수s5', lambda x : np.std(x)/np.mean(x))]).reset_index().fillna(0)


X= pd.merge(X, object60, on = '출신대학s5_지역', how='left')

### 7. 대학전공s5_대분류 + 근무경력

In [133]:
object6 = X.groupby('대학전공s5_대분류')['근무경력'].agg([('대학전공s5_대분류_총근무경력s5', np.sum),
                                           #  ('구매건수', np.size),
                                            ('대학전공s5_대분류_평균근무경력s5', np.mean),
                                            ('대학전공s5_대분류_최대근무경력s5', np.max),
                                            ('대학전공s5_대분류_최소근무경력s5', np.min),
                                            ('대학전공s5_대분류_근무경력표준편차s5', np.std),   
                                            ('대학전공s5_대분류_근무경력변동계수s5', lambda x : np.std(x)/np.mean(x))]).reset_index().fillna(0)


X= pd.merge(X, object6, on = '대학전공s5_대분류', how='left')

### 8. 대학전공s5_소분류 + 근무경력

In [134]:
object7 = X.groupby('대학전공s5_소분류')['근무경력'].agg([('대학전공s5_소분류_총근무경력s5', np.sum),
                                           #  ('구매건수', np.size),
                                            ('대학전공s5_소분류_평균근무경력s5', np.mean),
                                            ('대학전공s5_소분류_최대근무경력s5', np.max),
                                            ('대학전공s5_소분류_최소근무경력s5', np.min),
                                            ('대학전공s5_소분류_근무경력표준편차s5', np.std),   
                                            ('대학전공s5_소분류_근무경력변동계수s5', lambda x : np.std(x)/np.mean(x))]).reset_index().fillna(0)


X= pd.merge(X, object7, on = '대학전공s5_소분류', how='left')

### 9. 출신대학_지역 + 대학성적

In [135]:
object8 = X.groupby('출신대학s5_지역')['대학성적'].agg([('출신대학_지역_총대학성적s5', np.sum),
                                           #  ('구매건수', np.size),
                                            ('출신대학s5_지역_평균대학성적s5', np.mean),
                                            ('출신대학s5_지역_최대대학성적s5', np.max),
                                            ('출신대학s5_지역_최소대학성적s5', np.min),
                                            ('출신대학s5_지역_대학성적표준편차s5', np.std),   
                                            ('출신대학s5_지역_대학성적변동계수s5', lambda x : np.std(x)/np.mean(x))]).reset_index().fillna(0)


X= pd.merge(X, object8, on = '출신대학s5_지역', how='left')

### 10. 대학전공s5_대분류 + 대학성적

In [136]:
object9 = X.groupby('대학전공s5_대분류')['대학성적'].agg([('대학전공s5_대분류_총대학성적s5', np.sum),
                                           #  ('구매건수', np.size),
                                            ('대학전공s5_대분류_평균대학성적s5', np.mean),
                                            ('대학전공s5_대분류_최대대학성적s5', np.max),
                                            ('대학전공s5_대분류_최소대학성적s5', np.min),
                                            ('대학전공s5_대분류_대학성적표준편차s5', np.std),   
                                            ('대학전공s5_대분류_대학성적변동계수s5', lambda x : np.std(x)/np.mean(x))]).reset_index().fillna(0)


X= pd.merge(X, object9, on = '대학전공s5_대분류', how='left')

### 11. 직무태그_1 + 근무경력

In [137]:
object10 = X.groupby('직무태그_1')['근무경력'].agg([('직무태그_1_총근무경력s5', np.sum),
                                           #  ('구매건수', np.size),
                                            ('직무태그_1_평균근무경력s5', np.mean),
                                            ('직무태그_1_최대근무경력s5', np.max),
                                            ('직무태그_1_최소근무경력s5', np.min),
                                            ('직무태그_1_근무경력표준편차s5', np.std),   
                                            ('직무태그_1_근무경력변동계수s5', lambda x : np.std(x)/np.mean(x))]).reset_index().fillna(0)


X= pd.merge(X, object10, on = '직무태그_1', how='left')

### 12. 직무태그_1 + 대학성적

In [138]:
object11 = X.groupby('직무태그_1')['대학성적'].agg([('직무태그_1_총대학성적s5', np.sum),
                                           #  ('구매건수', np.size),
                                            ('직무태그_1_평균대학성적s5', np.mean),
                                            ('직무태그_1_최대대학성적s5', np.max),
                                            ('직무태그_1_최소대학성적s5', np.min),
                                            ('직무태그_1_대학성적표준편차s5', np.std),   
                                            ('직무태그_1_대학성적변동계수s5', lambda x : np.std(x)/np.mean(x))]).reset_index().fillna(0)


X= pd.merge(X, object11, on = '직무태그_1', how='left')

### 13. 직종 + 근무경력

In [139]:
object12 = X.groupby('직종')['근무경력'].agg([('직종_총근무경력s5', np.sum),
                                           #  ('구매건수', np.size),
                                            ('직종_평균근무경력s5', np.mean),
                                            ('직종_최대근무경력s5', np.max),
                                            ('직종_최소근무경력s5', np.min),
                                            ('직종_근무경력표준편차s5', np.std),   
                                            ('직종_근무경력변동계수s5', lambda x : np.std(x)/np.mean(x))]).reset_index().fillna(0)


X= pd.merge(X, object12, on = '직종', how='left')

### 14. 직종 + 대학성적

In [140]:
object13 = X.groupby('직종')['대학성적'].agg([('직종_총대학성적s5', np.sum),
                                           #  ('구매건수', np.size),
                                            ('직종_평균대학성적s5', np.mean),
                                            ('직종_최대대학성적s5', np.max),
                                            ('직종_최소대학성적s5', np.min),
                                            ('직종_대학성적표준편차s5', np.std),   
                                            ('직종_대학성적변동계수s5', lambda x : np.std(x)/np.mean(x))]).reset_index().fillna(0)


X= pd.merge(X, object13, on = '직종', how='left')

In [141]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27618 entries, 0 to 27617
Columns: 102 entries, 직종 to 직종_대학성적변동계수s5
dtypes: float64(64), int64(24), object(14)
memory usage: 21.7+ MB


### 15. 근무지역_소분류 + 근무경력

In [142]:
object14 = X.groupby('근무지역_소분류')['근무경력'].agg([('근무지역_소분류_총근무경력s5', np.sum),
                                           #  ('구매건수', np.size),
                                            ('근무지역_소분류_평균근무경력s5', np.mean),
                                            ('근무지역_소분류_최대근무경력s5', np.max),
                                            ('근무지역_소분류_최소근무경력s5', np.min),
                                            ('근무지역_소분류_근무경력표준편차s5', np.std),   
                                            ('근무지역_소분류_근무경력변동계수s5', lambda x : np.std(x)/np.mean(x))]).reset_index().fillna(0)


X= pd.merge(X, object14, on = '근무지역_소분류', how='left')

### 16. 근무지역_대분류 + 근무경력

In [143]:
object15 = X.groupby('근무지역_대분류')['근무경력'].agg([('근무지역_대분류_총근무경력s5', np.sum),
                                           #  ('구매건수', np.size),
                                            ('근무지역_대분류_평균근무경력s5', np.mean),
                                            ('근무지역_대분류_최대근무경력s5', np.max),
                                            ('근무지역_대분류_최소근무경력s5', np.min),
                                            ('근무지역_대분류_근무경력표준편차s5', np.std),   
                                            ('근무지역_대분류_근무경력변동계수s5', lambda x : np.std(x)/np.mean(x))]).reset_index().fillna(0)


X= pd.merge(X, object15, on = '근무지역_대분류', how='left')

### 17. 근무지역_소분류 + 대학성적

In [144]:
object17 = X.groupby('근무지역_소분류')['대학성적'].agg([('근무지역_소분류_총대학성적s5', np.sum),
                                           #  ('구매건수', np.size),
                                            ('근무지역_소분류_평균대학성적s5', np.mean),
                                            ('근무지역_소분류_최대대학성적s5', np.max),
                                            ('근무지역_소분류_최소대학성적s5', np.min),
                                            ('근무지역_소분류_대학성적표준편차s5', np.std),   
                                            ('근무지역_소분류_대학성적변동계수s5', lambda x : np.std(x)/np.mean(x))]).reset_index().fillna(0)


X= pd.merge(X, object17, on = '근무지역_소분류', how='left')

### 18. 근무지역_대분류 + 대학성적

In [145]:
object18 = X.groupby('근무지역_대분류')['대학성적'].agg([('근무지역_대분류_총대학성적s5', np.sum),
                                           #  ('구매건수', np.size),
                                            ('근무지역_대분류_평균대학성적s5', np.mean),
                                            ('근무지역_대분류_최대대학성적s5', np.max),
                                            ('근무지역_대분류_최소대학성적s5', np.min),
                                            ('근무지역_대분류_대학성적표준편차s5', np.std),   
                                            ('근무지역_대분류_대학성적변동계수s5', lambda x : np.std(x)/np.mean(x))]).reset_index().fillna(0)


X= pd.merge(X, object18, on = '근무지역_대분류', how='left')

# (3) Addition or Subtraction features

# Product Features

In [146]:
X['근무경력_x_대학성적'] = X['근무경력'] * X['대학성적']

In [147]:
X['근무형태_총근무경력s5x근무경력_x_대학성적'] = X['근무형태_총근무경력s5'] * X['근무경력_x_대학성적']

In [148]:
X['피처 중요도 상위 top3'] = X['근무형태_총근무경력s5'] + X['근무형태_총근무경력s5'] + X['직무태그_1_평균근무경력s5']

In [149]:
X['피처 중요도 상위 top3 가중치'] = X['근무형태_총근무경력s5']*4 + X['근무형태_총근무경력s5']*3 + X['직무태그_1_평균근무경력s5']*2

In [150]:
X = X.drop(['직종','세부직종','직무태그','근무경력','근무형태','근무지역','출신대학','어학시험', '자격증', '대학성적'], axis=1)

In [151]:
X_train5 = X.iloc[:X_train5.shape[0],:].reset_index(drop=True)
X_test5 = X.iloc[X_train5.shape[0]:,:].reset_index(drop=True)

In [161]:
feature_resume_table = resumetable(X)
feature_resume_table

데이터셋 형상: (27618, 120)


,피처,데이터 타입,결측값 개수,고윳값 개수,첫 번째 값,두 번째 값,세 번째 값
0,대학전공,object,0,414,국문,관광,정보
1,대학전공s5_대분류,object,0,19,어문대학,경영학과,소프트웨어대학
2,대학전공s5_소분류,object,0,181,국어국문학과,안경학과,신학과
3,출신대학s5_지역,object,0,4,서울,지방,경기
4,직무태그_1,object,0,1484,취재,없음,하드
5,근무지역_소분류,object,0,76,서울,해외,서울
6,근무지역_대분류,object,0,26,수도권,해외,수도권
7,직무태그글자수범위별,int64,0,8,40,10,70
8,근무형태_총근무경력s5,int64,0,28,200114,653277,200114
9,근무형태_평균근무경력s5,float64,0,28,12.050705,69.438457,12.050705


In [240]:
X_train5.to_csv('은디디feature_selected_train.csv', index=False,encoding ='utf-8')
X_test5.to_csv('은디디feature_selected_test.csv', index=False,encoding = 'utf-8')